In [ ]:
import os
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torch.optim import AdamW
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from sklearn.model_selection import train_test_split
from transformers.models.bert.modeling_bert import BertEncoder
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from functools import partial
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig

##Данные

In [ ]:
# Загрузка и предобработка данных IMDB
file_path = "IMDB Dataset.csv"
df = pd.read_csv(file_path)
df['sentiment'] = df['sentiment'].map({'negative': 0, 'positive': 1})
df = df.head(1000)

In [ ]:
# Разбиваем данные на train, validation, test
train_val_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42)
print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

Train size: 560
Validation size: 140
Test size: 300


In [ ]:
#Загрузка токенизатора
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
#Инициализация, макс длина берта - 512 токенов
class IMDBDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        review = self.data.iloc[idx]['review']
        label = self.data.iloc[idx]['sentiment']
        encoding = self.tokenizer(
            review,
            #дополняем последовательность до 512 токенов
            padding="max_length",
            #обрезаем до 512 токенов
            truncation=True,
            max_length=self.max_length,
            #возвращаем тензоры pytorch
            return_tensors="pt"
        )
        #squeeze(0) удаляет лишнюю размерность
        item = {key: tensor.squeeze(0) for key, tensor in encoding.items()}
        item['labels'] = torch.tensor(label, dtype=torch.long)
        return item

In [ ]:
# Создаем объекты датасетов и DataLoaderов
train_dataset = IMDBDataset(train_df, tokenizer)
val_dataset = IMDBDataset(val_df, tokenizer)
test_dataset = IMDBDataset(test_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##BANANAS

Для начала необходимо сгенерировать случайные конфигурации берта (различное количество слоев). 1 - слой активен, 0 - слой неактивен в конфигурации

In [ ]:
#Генерация модельных конфигураций случайной маски
def generate_random_config(num_layers=12, min_layers=4, max_layers=12):
    config = [0] * num_layers
    selected = random.randint(min_layers, max_layers)
    positions = random.sample(range(num_layers), selected)
    for pos in positions:
        config[pos] = 1
    return config

Способ оптимизации NAS, предложенный в статье - мутация слоев. Каждый слой изменяет свой статус (активен/неактивен) с 30% вероятностью. При это отслеживаем, что кол-во слоев не должно быть меньше 4 (сильно падает accuracy) и не больше 12 (цель - уменьшить кол-во слоев в BERT без потери качества)

In [ ]:
#Модификация конфигураций
def modify_config(config: list, change_prob=0.3, min_layers=4, max_layers=12):
    updated_config = config.copy()
    for i in range(len(updated_config)):
        if random.random() < change_prob:
            updated_config[i] = 1 - updated_config[i]
    active_layers = sum(updated_config)
    if active_layers < min_layers:
        inactive_indices = [i for i, val in enumerate(updated_config) if val == 0]
        random.shuffle(inactive_indices)
        for i in inactive_indices[:(min_layers - active_layers)]:
            updated_config[i] = 1
    elif active_layers > max_layers:
        active_indices = [i for i, val in enumerate(updated_config) if val == 1]
        random.shuffle(active_indices)
        for i in active_indices[:(active_layers - max_layers)]:
            updated_config[i] = 0
    return updated_config

Создание модели BERT с выбранными конфигурациями (слоями)

In [ ]:
#Создание моделей по конфигурациям
def create_model_from_config(config: list, original_model: BertForSequenceClassification):
    new_model = copy.deepcopy(original_model)
    all_modules = new_model.bert.encoder.layer
    selected_modules = nn.ModuleList([module for module, flag in zip(all_modules, config) if flag == 1])
    if len(selected_modules) == 0:
        selected_modules = nn.ModuleList([all_modules[0]])
    new_model.bert.encoder.layer = selected_modules
    new_model.config.num_hidden_layers = len(selected_modules)
    print(f"Создана модель с конфигурацией: {config} , содержит {len(selected_modules)} слоёв (из {len(all_modules)})")
    return new_model

Обучаем полученную конфигурации, смотрим на качество (accuracy) полученного BERTа

In [ ]:
#Оценка моделей
def calculate_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#оценка модели
def assess_model(config: list, original_model: BertForSequenceClassification,
                 train_loader: DataLoader, val_loader: DataLoader,
                 num_epochs=1, device=device,
                 min_score=0.75, max_params=1e8):
    model = create_model_from_config(config, original_model)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)
    scaler = torch.cuda.amp.GradScaler()

    model.train()
    for epoch in range(num_epochs):
        for batch in tqdm(train_loader, desc=f"Обучение модели (config={config})", leave=False):
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(**inputs, labels=labels)
                loss = outputs.loss
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        torch.cuda.empty_cache()

    model.eval()
    total_samples, correct_predictions = 0, 0
    for batch in tqdm(val_loader, desc=f"Оценка модели (config={config})", leave=False):
        inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)
        with torch.cuda.amp.autocast():
            outputs = model(**inputs)
        predictions = torch.argmax(F.softmax(outputs.logits, dim=1), dim=1)
        total_samples += labels.size(0)
        correct_predictions += (predictions == labels).sum().item()

    accuracy = correct_predictions / total_samples if total_samples > 0 else 0.0
    param_count = calculate_params(model)
    if accuracy < min_score:
        print(f"Архитектура отклонена: точность {accuracy:.4f} < минимального порога {min_score}")
        result = -float('inf')
    elif param_count > max_params:
        print(f"Архитектура отклонена: параметров {param_count} > максимального порога {max_params}")
        result = -float('inf')
    else:
        lambda_val = 1e-8
        result = accuracy - lambda_val * param_count
    print(f"Конфигурация (config = {config}): точность: {accuracy:.4f}, параметров: {param_count}, метрика: {result:.6f}")
    return result, accuracy, param_count

Для реализации BANANAS нужен предиктор, который будет говорить, насколько хороша конфигурация BERTа без полного обучения.

In [ ]:
#Предсказательная модель
class RewardPredictor(nn.Module):
    def __init__(self, input_size=12):
        super(RewardPredictor, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.layers(x)

Компонент алгоритма BANANAS - обучение ансамбля нейросетей-предсказателей, который ускоряет нейроархитектурный поиск. Ансамбль даёт разные предсказания для одной конфигурации, выбираем кандидата с максимальным средним ± std

In [ ]:
#Обучение ансамбля предсказателей
def train_predictors(history_data, predictors, num_epochs=100, learning_rate=1e-3):
    features = []
    targets = []
    for record in history_data:
        config, result, _, _ = record
        if np.isfinite(result):
            features.append(config)
            targets.append(result)
    if len(features) == 0:
        print("Недостаточно данных для обучения предсказателей.")
        return
    features = torch.tensor(features, dtype=torch.float32)
    targets = torch.tensor(targets, dtype=torch.float32).unsqueeze(1)

    loss_fn = nn.MSELoss()
    optimizers = [AdamW(model.parameters(), lr=learning_rate) for model in predictors]
    batch_size = 4
    for epoch in range(num_epochs):
        indices = torch.randperm(features.size(0))
        for i in range(0, features.size(0), batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_x = features[batch_indices].to(device)
            batch_y = targets[batch_indices].to(device)
            for model, optimizer in zip(predictors, optimizers):
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = loss_fn(outputs, batch_y)
                loss.backward()
                optimizer.step()


 Стохастическое предсказание метрики качества (reward) для новой конфигурации модели (нужно для оценки конфигурации без её обучения)

In [ ]:
# предсказание значения метрики
def predict_reward(model_config, predictors):
    config_tensor = torch.tensor(model_config, dtype=torch.float32).unsqueeze(0).to(device)
    predictions = []
    for model in predictors:
        with torch.no_grad():
            pred = model(config_tensor).item()
        predictions.append(pred)
    mean_pred = np.mean(predictions)
    std_pred = np.std(predictions)
    if std_pred == 0:
        return mean_pred
    sample = np.random.normal(mean_pred, std_pred)
    return sample

Отбираем лучшие конфигурации из всех рассмотренныхи и мутируем их

In [ ]:
# генерация новых конфигураций
def generate_new_configs(history_data, num_configs=10, change_prob=0.3, min_layers=4, max_layers=12):
    sorted_history = sorted(history_data, key=lambda x: x[1], reverse=True)
    top_configs = [entry[0] for entry in sorted_history[:max(1, len(sorted_history)//2)]]
    new_configs = []
    while len(new_configs) < num_configs:
        base_config = random.choice(top_configs)
        modified_config = modify_config(base_config, change_prob=change_prob, min_layers=min_layers, max_layers=max_layers)
        new_configs.append(modified_config)
    return new_configs

BANANAS

In [ ]:
def run_architecture_search():
    base_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
    base_model.to(device)

    # Параметры поиска
    total_layers = 12
    min_active = 4
    max_active = 12
    initial_samples = 5   # число начальных случайных оценок
    total_iterations = 10   # общее число итераций
    num_predictors = 3
    configs_per_iteration = 10
    predictor_epochs = 50
    predictor_learning_rate = 1e-3

    search_history = []

    print("Начальный этап случайного поиска:")
    for i in range(initial_samples):
        config = generate_random_config(total_layers, min_active, max_active)
        result, accuracy, params = assess_model(config, base_model, train_loader, val_loader,
                                               num_epochs=1, device=device,
                                               min_score=0.55, max_params=1e8)
        search_history.append((config, result, accuracy, params))
        print(f"Начальная конфигурация {i+1}: config={config}, метрика={result:.6f}, точность={accuracy:.4f}, параметры={params}")

    best_config = None
    best_result = -float('inf')

    #инициализация предсказателей
    predictors = [RewardPredictor(input_size=total_layers).to(device) for _ in range(num_predictors)]

    print("\nОсновной этап поиска архитектур:")
    for iteration in range(initial_samples, total_iterations):
        print(f"\nИтерация {iteration+1}")
        # обучение предсказателей
        train_predictors(search_history, predictors, num_epochs=predictor_epochs, learning_rate=predictor_learning_rate)

        # генерация новых конфигураций
        candidate_configs = generate_new_configs(search_history, num_configs=configs_per_iteration,
                                              change_prob=0.3, min_layers=min_active, max_layers=max_active)

        # оценка кандидатов
        predicted_scores = []
        for candidate in candidate_configs:
            score = predict_reward(candidate, predictors)
            predicted_scores.append(score)
            print(f"Конфигурация: {candidate}, предсказанная метрика: {score:.6f}")

        # выбор лучшей конфигурации
        best_candidate_idx = np.argmax(predicted_scores)
        selected_config = candidate_configs[best_candidate_idx]
        print(f"Выбрана конфигурация: {selected_config} с предсказанной метрикой {predicted_scores[best_candidate_idx]:.6f}")

        # оценка выбранной конфигурации
        result, accuracy, params = assess_model(selected_config, base_model, train_loader, val_loader,
                                               num_epochs=1, device=device,
                                               min_score=0.55, max_params=1e8)
        print(f"Оценка конфигурации: config={selected_config}, метрика={result:.6f}, точность={accuracy:.4f}, параметры={params}")
        search_history.append((selected_config, result, accuracy, params))

        if result > best_result:
            best_result = result
            best_config = selected_config.copy()
            print(f"Новая лучшая конфигурация: config={best_config} с метрикой {best_result:.6f}")

    print("\n=== Поиск оптимальной архитектуры завершен ===")
    print(f"Лучшая конфигурация: {best_config}")
    print(f"Лучшее значение метрики: {best_result:.6f}")
    print(f"Точность модели: {accuracy:.4f}")
    print(f"Количество параметров: {params}")

if __name__ == "__main__":
    run_architecture_search()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Начальный этап случайного поиска:
Создана модель с конфигурацией: [0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0] , содержит 5 слоёв (из 12)


<ipython-input-52-58441a36b5ad>:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Обучение модели (config=[0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]):   0%|          | 0/35 [00:00<?, ?it/s]

<ipython-input-52-58441a36b5ad>:21: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


...

Оценка модели (config=[1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]):   0%|          | 0/9 [00:00<?, ?it/s]

Конфигурация (config = [1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]): точность: 0.5857, параметров: 52780802, метрика: 0.057906
Оценка конфигурации: config=[1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0], метрика=0.057906, точность=0.5857, параметры=52780802

Итерация 10
Конфигурация: [1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], предсказанная метрика: -0.045183
Конфигурация: [0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1], предсказанная метрика: -0.089197
Конфигурация: [0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1], предсказанная метрика: -0.079875
Конфигурация: [0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0], предсказанная метрика: -0.147243
Конфигурация: [0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1], предсказанная метрика: -0.031025
Конфигурация: [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1], предсказанная метрика: -0.145677
Конфигурация: [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1], предсказанная метрика: -0.086694
Конфигурация: [1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0], предсказанная метрика: -0.049333
Конфигурация: [1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0], предсказанная м

Обучение модели (config=[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0]):   0%|          | 0/35 [00:00<?, ?it/s]

Оценка модели (config=[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0]):   0%|          | 0/9 [00:00<?, ?it/s]

Конфигурация (config = [0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0]): точность: 0.6000, параметров: 52780802, метрика: 0.072192
Оценка конфигурации: config=[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0], метрика=0.072192, точность=0.6000, параметры=52780802

=== Поиск оптимальной архитектуры завершен ===
Лучшая конфигурация: [1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
Лучшее значение метрики: 0.093621
Точность модели: 0.6000
Количество параметров: 52780802
